In [1]:
# https://www.analyticsvidhya.com/blog/2018/09/multivariate-time-series-guide-forecasting-modeling-python-codes/
#import required packages
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#read the data
# df = pd.read_csv("AirQualityUCI.csv")
df = pd.read_csv("AirQualityUCI1.csv", parse_dates=[['Date', 'Time']])

#check the dtypes
df.dtypes

Date_Time        datetime64[ns]
CO(GT)                  float64
PT08.S1(CO)               int64
NMHC(GT)                  int64
C6H6(GT)                float64
PT08.S2(NMHC)             int64
NOx(GT)                   int64
PT08.S3(NOx)              int64
NO2(GT)                   int64
PT08.S4(NO2)              int64
PT08.S5(O3)               int64
T                       float64
RH                      float64
AH                      float64
dtype: object

In [2]:
df['Date_Time'] = pd.to_datetime(df.Date_Time , format = '%d/%m/%Y %H:%M:%S')
data = df.drop(['Date_Time'], axis=1)
data.index = df.Date_Time

In [3]:
#missing value treatment
cols = data.columns
for j in cols:
    for i in range(0,len(data)):
       if data[j][i] == -200:
           data[j][i] = data[j][i-1]

#checking stationarity
from statsmodels.tsa.vector_ar.vecm import coint_johansen
#since the test works for only 12 variables, I have randomly dropped
#in the next iteration, I would drop another and check the eigenvalues
johan_test_temp = data.drop([ 'CO(GT)'], axis=1)
coint_johansen(johan_test_temp,-1,1).eig

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


array([1.78898435e-01, 1.57134377e-01, 1.18954137e-01, 1.06959373e-01,
       9.46073557e-02, 7.07957006e-02, 6.01068434e-02, 3.50329354e-02,
       3.15883669e-02, 1.22674586e-02, 2.57530260e-03, 7.45284097e-05])

In [4]:
#creating the train and validation set
train = data[:int(0.8*(len(data)))]
valid = data[int(0.8*(len(data))):]

#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(endog=train)
model_fit = model.fit()

# make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(valid))

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [9]:
#converting predictions to dataframe
pred = pd.DataFrame(index=range(0,len(prediction)),columns=[cols])
for j in range(0,13):
    for i in range(0, len(prediction)):
       pred.iloc[i][j] = prediction[i][j]

import numpy as np 
from sklearn.metrics import mean_squared_error

#check rmse
for i in cols:
    print('rmse value for', i, 'is : ', np.sqrt(mean_squared_error(pred[i], valid[i])))

rmse value for CO(GT) is :  1.420578857136888
rmse value for PT08.S1(CO) is :  207.5085346586845
rmse value for NMHC(GT) is :  5.933350435673447
rmse value for C6H6(GT) is :  7.216452913975654
rmse value for PT08.S2(NMHC) is :  280.8235876455336
rmse value for NOx(GT) is :  216.8821203438309
rmse value for PT08.S3(NOx) is :  247.14794891174472
rmse value for NO2(GT) is :  66.97486911409179
rmse value for PT08.S4(NO2) is :  491.8979178585775
rmse value for PT08.S5(O3) is :  449.1470603347469
rmse value for T is :  10.754818832134179
rmse value for RH is :  17.173164219187406
rmse value for AH is :  0.5209713857455913


In [10]:
#make final predictions
model = VAR(endog=data)
model_fit = model.fit()
yhat = model_fit.forecast(model_fit.y, steps=1)
print(yhat)

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


[[2.33299697e+00 1.08513419e+03 2.80615080e+02 1.23632702e+01
  1.05357764e+03 2.80662482e+02 6.61083679e+02 1.68278272e+02
  1.15778550e+03 8.49851767e+02 2.73294054e+01 1.55977974e+01
  5.15400974e-01]]
